In [22]:
import numpy as np
import re

a3m_fn = "/public_data/ml/CATH40/CATH40-20JUN08/msa/12asA00.a3m"
ccm_fn = "/public_data/ml/CATH40/CATH40-20JUN08/ccm/12asA00.npy"
aa_code = "ARNDCEQGHILKMFPSTWYVX-"

mapping = {}
for i, char in enumerate(aa_code):
    mapping[char] = i

# TODO: Calculate sequence profile feature (amino acid propensity per position)

In [23]:
# 1. Read a3m file (ingnore lower case - insertion)
prot_seqs = []
with open(a3m_fn, 'r') as a3m_f:
    for line in a3m_f:
        if not line.startswith('>'):
            prot_seqs.append(np.array([mapping.get(res) for res in re.sub(f'[^{aa_code}]', "", line)]))

prot_align = np.transpose(np.vstack(prot_seqs))

In [24]:
# 2. Calculate sequence profile (N_res, 20+1+1) 20 standard a.a.(ARNDCQEGHILKMFPSTWVU) + 1 unknown('X') + 1 gap('-')
seq_profile = np.array([np.bincount(res, minlength=22) for res in prot_align])/prot_align.shape[1]

In [25]:
# 3. Tile sequence profile feature to make it as 2D(N_res, N_res, 22) * 2 (x-tile, y-tile)
N_res = len(prot_align)
tiled_prof = np.tile(seq_profile, (N_res, 1)).reshape(N_res, N_res, 22)
final_prof = np.concatenate((tiled_prof, np.rot90(tiled_prof)), axis=2)

In [26]:
# 4. concat to CCM feature (N_res, N_res, 441 + 22 + 22)
ccm = np.load(ccm_fn)
final_feature = np.concatenate((final_prof, ccm), axis=2)
print(final_feature.shape)

(327, 327, 485)


In [27]:
print(seq_profile[10])

[0.05900484 0.03038309 0.01100837 0.03786878 0.00396301 0.06428886
 0.05812417 0.00528402 0.01056803 0.02906209 0.17217085 0.12813738
 0.04051079 0.16336416 0.         0.01981506 0.05768384 0.001321
 0.08410392 0.01541171 0.         0.00792602]
